## Imageomics Bioclip-demo 
BioClip is a model using CLIP architecture as a vison model for general organismal biology. Trained on TreeOfLife-10M dataset. BioClip includes a understanding on the hierarchical structure that relates species across the tree of life.

For the purpose of Plant Commnicator I will utilize this model to identify common houseplants

## 1. Load the Model and Tokenizer
To begin, we will install the necessary libraries for the model and houseplant identification task. We will load a pre-trained model and its corresponding tokenizer using the Hugging Face Transformers library. The tokenizer is responsible for converting the input text into a format that the model can understand, while the model is used to perform the actual predictions or classifications. We will specify the model name, load the tokenizer, and then load the model. This process ensures that we have all the necessary components to perform text processing and analysis.

In [3]:
import open_clip
import torch
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import pandas as pd

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
tokenizer = open_clip.get_tokenizer('hf-hub:imageomics/bioclip')

## 2. Tokenize Common houseplant names

We will input a file that includes a list of common houseplant names included in the BioClip model. This will be used to classify and tokenize the ext using the model's tokenizer. This prepares the items for input into the model by converting them into a format the model can process.

In [4]:
# Load the CSV file
df = pd.read_csv("./houseplants.csv")  # Ensure the path is correct

# Extract plant names (common or species names depending on the column names)
plant_names = df["Common Name"].tolist()

tokenized_names = tokenizer(plant_names)

print("Tokenized Plant Names:", tokenized_names)

Tokenized Plant Names: tensor([[49406,  8798,  3912,  ...,     0,     0,     0],
        [49406,  3021, 10647,  ...,     0,     0,     0],
        [49406,   628, 41965,  ...,     0,     0,     0],
        ...,
        [49406,  3329,   539,  ...,     0,     0,     0],
        [49406,  9287,  4108,  ...,     0,     0,     0],
        [49406,  1192,   917,  ...,     0,     0,     0]])


In [ ]:
image_path="./images/RubberTreePlant.webp"
image = preprocess_val(Image.open(image_path)).unsqueeze(0)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image = image.to(device)

# Get text embeddings
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(tokenized_names)

# Compute similarity
similarities = torch.cosine_similarity(image_features, text_features)
predicted_index = similarities.argmax().item()
predicted_plant = plant_names[predicted_index]

print("Predicted Plant:", predicted_plant)

Predicted Plant: Pothos


## CLIP output and accuracy
When given a csv file with names of common household plant names and an input of a plant included in the list the output results are incorrect. It outputs Pothos, which is not the intended result.

## Gradio client api
The next method of producing the desired output when given a image file is through the  gradio client api method. This is the recommended api to use for the bioclip model. 

In [ ]:
from gradio_client import Client, handle_file

client = Client("imageomics/bioclip-demo")
result = client.predict(
		img=handle_file('./images/RubberTreePlant.webp'),
		rank="Species",
		api_name="/lambda"
)
print(result)

Loaded as API: https://imageomics-bioclip-demo.hf.space ✔
({'label': 'Plantae Tracheophyta Magnoliopsida Rosales Moraceae Ficus elastica (Indian rubber fig)', 'confidences': [{'label': 'Plantae Tracheophyta Magnoliopsida Rosales Moraceae Ficus elastica (Indian rubber fig)', 'confidence': 0.8459776043891907}, {'label': 'Plantae Tracheophyta Magnoliopsida Ericales Marcgraviaceae Souroubea pachyphylla', 'confidence': 0.053483083844184875}, {'label': 'Plantae Tracheophyta Magnoliopsida Ericales Ericaceae Rhododendron cuneatum', 'confidence': 0.02335861325263977}, {'label': 'Plantae Tracheophyta Magnoliopsida Asparagales Orchidaceae Bulbophyllum rufilabrum', 'confidence': 0.010561013594269753}, {'label': 'Plantae Tracheophyta Magnoliopsida Ericales Sapotaceae Manilkara rufula', 'confidence': 0.009562669321894646}]}, 'C:\\Users\\Imbat\\AppData\\Local\\Temp\\gradio\\6ee223c4639b8a15d2714acc593920280834c7f1f306134d070f15b2fd477d03\\image.webp', "<p>Check out the EOL entry for Plantae Tracheoph

This code produces the correct output since it is a direct output of the model. Going forward the app will use this api

# Plant Disease Detection Workflow

In this section, we will explore different models to identify the best one for detecting plant diseases. The workflow includes loading the necessary models and processors, asking relevant questions about the plant's health, and analyzing the responses to determine the condition of the plant. This process will help in diagnosing any potential issues and providing appropriate care recommendations.

We are using the BLIP (Bootstrapping Language-Image Pre-training) model from Hugging Face. BLIP is designed for vision-language tasks, such as visual question answering (VQA). It leverages both visual and textual information to generate accurate responses to questions about images. By using this model, we can ask specific questions about the plant's health and obtain detailed answers based on the visual input.

Loading the model

In [1]:
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import torch
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

if not huggingface_token:
    raise ValueError("Hugging Face token is missing. Please set the HUGGINGFACE_TOKEN environment variable.")

# Load model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base", token=huggingface_token) 
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base", token=huggingface_token)




inputting an image of a plant with a question regarding physical appearance

In [ ]:
# Load an image of a plant
image = Image.open("images/RubberTreePlant.webp").convert("RGB")

# Ask a question
question = "Are there any yellow or brown spots on the leaves?"
inputs = processor(image, question, return_tensors="pt")

# Generate answer
output = model.generate(**inputs)
answer = processor.decode(output[0], skip_special_tokens=True)

print(answer)

no


developing new methods to determine severity of plant condition to limit the model's use

In [ ]:
# Load an image of a plant
image = Image.open("images/dying-plant.webp").convert("RGB")

question = "on a scale of 1-10 whats the health condition of this plant?"

inputs = processor(image, question, return_tensors="pt")

# Generate answer
output = model.generate(**inputs)
answer = processor.decode(output[0], skip_special_tokens=True)

print(answer)

healthy


asking the model multiple diagnostic questions on the plant's condition

In [ ]:
plant_health_questions = {
    "leaf_color": [
        "What color are the leaves?",
        "Are there any yellow, brown, or black spots on the leaves?",
        "Do the leaves look pale or discolored?"
    ],
    "leaf_texture": [
        "Are the leaves curled, wrinkled, or distorted?",
        "Do the leaves have holes or bite marks?",
        "Are the leaves wilted or drooping?"
    ],
    "stem_health": [
        "Is the plant stem firm or soft and mushy?",
        "Are there any black or dark spots on the stem?",
        "Are there any fungi or mold around the base of the plant?"
    ],
    "pest_damage": [
        "Are there any small insects visible on the leaves or stem?",
        "Are there any white, web-like substances on the plant?",
        "Do the leaves have tiny dots or specks?"
    ],
}

for questionCategory, questions in plant_health_questions.items():
    print(f"Plant Health Category: {questionCategory}")
    for question in questions:
        inputs = processor(image, question, return_tensors="pt")
        output = model.generate(**inputs)
        answer = processor.decode(output[0], skip_special_tokens=True)
        print(question, ":", answer)


What color are the leaves? : green
Are there any yellow, brown, or black spots on the leaves? : brown
Do the leaves look pale or discolored? : green
Are the leaves curled, wrinkled, or distorted? : curved
Do the leaves have holes or bite marks? : yes
Are the leaves wilted or drooping? : wilted
Is the plant stem firm or soft and mushy? : firm
Are there any black or dark spots on the stem? : black
Are there any fungi or mold around the base of the plant? : no
Are there any small insects visible on the leaves or stem? : yes
Are there any white, web-like substances on the plant? : no
Do the leaves have tiny dots or specks? : yes
Is the soil dry or moist? : dry
Does the soil appear waterlogged or compacted? : no
Are there any white deposits on the soil? : no


In [11]:
def analyze_plant_health(image_path):
    image = Image.open(image_path).convert("RGB")
    for questionCategory, questions in plant_health_questions.items():
        print(f"Plant Health Category: {questionCategory}")
        for question in questions:
            inputs = processor(image, question, return_tensors="pt")
            output = model.generate(**inputs)
            answer = processor.decode(output[0], skip_special_tokens=True)
            print(question, ":", answer)

In [ ]:
analyze_plant_health('images/RubberTreePlant.webp')

Plant Health Category: leaf_color
What color are the leaves? : green
Are there any yellow, brown, or black spots on the leaves? : no
Do the leaves look pale or discolored? : dark
Plant Health Category: leaf_texture
Are the leaves curled, wrinkled, or distorted? : curved
Do the leaves have holes or bite marks? : no
Are the leaves wilted or drooping? : wilted
Plant Health Category: stem_health
Is the plant stem firm or soft and mushy? : firm
Are there any black or dark spots on the stem? : dark
Are there any fungi or mold around the base of the plant? : no
Plant Health Category: pest_damage
Are there any small insects visible on the leaves or stem? : no
Are there any white, web-like substances on the plant? : no
Do the leaves have tiny dots or specks? : no
Plant Health Category: soil_conditions
Is the soil dry or moist? : moist
Does the soil appear waterlogged or compacted? : no
Are there any white deposits on the soil? : no


In [ ]:
plant_health_questions = {
    "Overwatering": [
        "Are there any white fuzzy spots?",
        "Are there any yellow, brown, or black spots on the leaves?",
    ],
    "Underwatering": [
        "Are the leaves wilting?",
        "Are the leaves crispy?",
        "Are the leaves dropping off?",
    ],
    "Low_light": [
        "Are the leaves yellow?",
        
    ],
    "High_light": [
        "Are the leaves sunburned?",
        "Are there any brown edges on the leaves?",
    ],
    "Nutrient_deficiency": [
        "Are the leaves turning brown?",
        "Are there any spots or patches on the leaves?",
        "Are the leaves curling?",
    ],
    "pest_damage": [
        "Are there any small insects visible on the plant?",
        "Are there any white, web-like substances on the plant?",
        "Do the leaves have tiny dots or specks?"
    ],
    "root_bound": [
        "Are the roots growing out of the pot?",
        "Is the plant pot too small for the plant?",
    ],
}

In [ ]:
def analyze_plant_health(image_path):
    image = Image.open(image_path).convert("RGB")
    for questionCategory, questions in plant_health_questions.items():
        print(f"Plant Health Category: {questionCategory}")
        for question in questions:
            inputs = processor(image, question, return_tensors="pt")
            output = model.generate(**inputs)
            answer = processor.decode(output[0], skip_special_tokens=True)
            print(question, ":", answer)

In [4]:
analyze_plant_health('images/RubberTreePlant.webp')

Plant Health Category: Overwatering
Are there any white fuzzy spots? : no
Are there any yellow, brown, or black spots on the leaves? : no
Plant Health Category: Underwatering
Are the leaves drooping or wilting? : wilting
Are the leaves crispy or dry? : dry
Plant Health Category: Low_light
Are the leaves small or sparse? : small
Are the leaves turning yellow or dropping off? : neither
Plant Health Category: High_light
Are the leaves scorched or sunburned? : no
Are there any brown edges on the leaves? : yes
Plant Health Category: Nutrient_deficiency
Are the leaves yellowing or turning brown? : turning brown
Are there any spots or patches on the leaves? : no
Are the leaves curling or twisting? : twisting
Plant Health Category: pest_damage
Are there any small insects visible on the leaves or stem? : no
Are there any white, web-like substances on the plant? : no
Do the leaves have tiny dots or specks? : no
Plant Health Category: root_bound
Are the roots growing out of the pot? : yes
Is the 

In [7]:
analyze_plant_health('images/dying-plant.webp')

Plant Health Category: Overwatering
Are there any white fuzzy spots? : no
Are there any yellow, brown, or black spots on the leaves? : no
Plant Health Category: Underwatering
Are the leaves wilting? : no
Are the leaves crispy? : no
Are the leaves dry? : no
Plant Health Category: Low_light
Are the leaves turning yellow? : no
Are the leaves dropping off? : yes
Plant Health Category: High_light
Are the leaves sunburned? : no
Are there any brown edges on the leaves? : no
Plant Health Category: Nutrient_deficiency
Are the leaves turning brown? : no
Are there any spots or patches on the leaves? : no
Are the leaves curling? : yes
Plant Health Category: pest_damage
Are there any small insects visible on the leaves or stem? : leaves
Are there any white, web-like substances on the plant? : no
Do the leaves have tiny dots or specks? : yes
Plant Health Category: root_bound
Are the roots growing out of the pot? : yes
Is the plant pot too small for the plant? : no


In [ ]:
def analyze_plant_health(image_path):
    image = Image.open(image_path).convert("RGB")
    for questionCategory, questions in plant_health_questions.items():
    #print(f"Plant Health Category: {questionCategory}")
        for question in questions:
            inputs = processor(image, question, return_tensors="pt")
            output = model.generate(**inputs)
            answer = processor.decode(output[0], skip_special_tokens=True)
            if(answer == "yes"):
                print(questionCategory, ":", question)
            

In [ ]:
analyze_plant_health('images/dying-plant.webp')

Low_light : Are the leaves dropping off?
Nutrient_deficiency : Are the leaves curling?
pest_damage : Do the leaves have tiny dots or specks?
root_bound : Are the roots growing out of the pot?


In [13]:
image_path = 'images/misshapen.jpg'
image = Image.open(image_path).convert("RGB")
question = "are the leaves misshapen?"
inputs = processor(image, question, return_tensors="pt")
output = model.generate(**inputs)
answer = processor.decode(output[0], skip_special_tokens=True)
print(answer)

no


In [18]:
analyze_plant_health('images/dying-plant.webp')
print("\n")
analyze_plant_health('images/dying-plant.webp')
print("\n")
analyze_plant_health('images/dying-plant.webp')
print("\n")

Low_light : Are the leaves dropping off?
Nutrient_deficiency : Are the leaves curling?
pest_damage : Do the leaves have tiny dots or specks?
root_bound : Are the roots growing out of the pot?


Low_light : Are the leaves dropping off?
Nutrient_deficiency : Are the leaves curling?
pest_damage : Do the leaves have tiny dots or specks?
root_bound : Are the roots growing out of the pot?


Low_light : Are the leaves dropping off?
Nutrient_deficiency : Are the leaves curling?
pest_damage : Do the leaves have tiny dots or specks?
root_bound : Are the roots growing out of the pot?




In [ ]:
def analyze_plant_health(image_path):
    image = Image.open(image_path).convert("RGB")
    for questionCategory, questions in plant_health_questions.items():
    #print(f"Plant Health Category: {questionCategory}")
        for question in questions:
            inputs = processor(image, question, return_tensors="pt")
            output = model.generate(**inputs)
            answer = processor.decode(output[0], skip_special_tokens=True)
            if(answer == "yes"):
                print(questionCategory, ":", question)